In [2]:
import retrieval


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tfidf = retrieval.TF_IDFSearch() # 리트리버 정의
tfidf.get_sparse_embedding() # 위키 기반의 tfidfvectorizer을 만들고 passge_embedding을 만듭니다.
tfidf.build_faiss() # faiss를 통해 빠른 서치가 가능하도록 faiss index를 만듭니다.
result = tfidf.search_query() # faiss를 사용하지 않고 tfidf를 통해 test dataset의 질문과 관련된 문서를 찾습니다.
result2 = tfidf.search_query_faiss() # faiss를 사용하여 tfidf를 통해 test dataset의 질문과 관련된 문서를 찾습니다.
result # result와 result2 는 같은 모양새로 반환됩니다.

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


56737
tfidf를 로드했습니다.
Faiss Indexer을 로드했습니다.


Sparse retrieval: 100%|██████████| 600/600 [00:00<00:00, 21784.07it/s]


쿼리당 20개의 문서를 faiss indexer을 통해 찾습니다.


Sparse retrieval: 100%|██████████| 600/600 [00:00<00:00, 24249.20it/s]


DatasetDict({
    validation: Dataset({
        features: ['context', 'id', 'question'],
        num_rows: 600
    })
})

In [3]:
from retrieval import Dense_embedding_retrieval, BM25Search
from MRC import Extraction_based_MRC

# Dense Embedding 모델 학습 후 저장
dense_retriever = Dense_embedding_retrieval()
dense_retriever.get_trainer()
dense_retriever.train()

[nltk_data] Downloading package punkt to
[nltk_data]     /data/ephemeral/home/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dens

Using cuda_amp half precision backend
***** Running training *****
  Num examples = 3952
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 2
  Total optimization steps = 741
  Number of trainable parameters = 336656384
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.230300,0.087784,0.966667,0.966488
2,0.095000,0.094672,0.970833,0.970819
3,0.017100,0.010370,1.000000,1.000000


***** Running Evaluation *****
  Num examples = 240
  Batch size = 8
Saving model checkpoint to ./Dense_embedding_retrieval_model_results/checkpoint-247
Configuration saved in ./Dense_embedding_retrieval_model_results/checkpoint-247/config.json
Model weights saved in ./Dense_embedding_retrieval_model_results/checkpoint-247/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 240
  Batch size = 8
Saving model checkpoint to ./Dense_embedding_retrieval_model_results/checkpoint-494
Configuration saved in ./Dense_embedding_retrieval_model_results/checkpoint-494/config.json
Model weights saved in ./Dense_embedding_retrieval_model_results/checkpoint-494/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 240
  Batch size = 8
Saving model checkpoint to ./Dense_embedding_retrieval_model_results/checkpoint-741
Configuration saved in ./Dense_embedding_retrieval_model_results/checkpoint-741/config.json
Model weights saved in ./Dense_embedding_retrieval_model_results/chec

In [4]:
# Dense Embedding 모델 학습 후 저장
from retrieval import BM25Search
bm25 = BM25Search()
bm25.get_sparse_embedding()

bm25_result = bm25.search_query_bm25()

Data route: /data/ephemeral/data/train_dataset
Test data route: /data/ephemeral/data/test_dataset
Wiki route: /data/ephemeral/data/wikipedia_documents.json
Data path: ./bm25_retrieval_result


/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (1131 > 512). Running this sequence through the model will result in indexing errors


File loaded successfully
위키 컨텍스트의 수: 56737
./bm25_retrieval_result 폴더가 없어서 만듭니다.
Passage embedding을 만듭니다.
임베딩을 피클 형태로 저장했습니다.


BM25 retrieval: 100%|██████████| 600/600 [00:00<00:00, 10526.51it/s]


In [5]:
# 리랭킹 과정
from retrieval import Dense_embedding_retrieval, BM25Search
from MRC import Extraction_based_MRC
import pickle
import os

dense_retriever = Dense_embedding_retrieval()
bm25 = BM25Search()

#Dense retriever 모델 로드 
dense_retriever.load_model()

# Reranking Retreiver 생성
reranked_result = dense_retriever.sequential_reranking(mode='test')

save_path = "./reranked/reranked_result.pkl"

os.makedirs(os.path.dirname(save_path), exist_ok=True)

# Pickle을 사용해 저장
with open(save_path, 'wb') as f:
    pickle.dump(reranked_result, f)

print(f"Reranked result saved to {save_path}")

[nltk_data] Downloading package punkt to
[nltk_data]     /data/ephemeral/home/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.de

Data route: /data/ephemeral/data/train_dataset
Test data route: /data/ephemeral/data/test_dataset
Wiki route: /data/ephemeral/data/wikipedia_documents.json
Data path: ./bm25_retrieval_result


Token indices sequence length is longer than the specified maximum sequence length for this model (1131 > 512). Running this sequence through the model will result in indexing errors


File loaded successfully
위키 컨텍스트의 수: 56737
제일 마지막 checkpoint: ./Dense_embedding_retrieval_model_results/checkpoint-741/trainer_state.json
best checkpoint: ./Dense_embedding_retrieval_model_results/checkpoint-741


Some weights of the model checkpoint at ./Dense_embedding_retrieval_model_results/checkpoint-741 were not used when initializing RobertaModel: ['q_model.bert.encoder.layer.8.attention.self.query.weight', 'p_model.bert.encoder.layer.10.intermediate.dense.weight', 'q_model.bert.encoder.layer.1.attention.self.query.bias', 'q_model.bert.embeddings.LayerNorm.weight', 'p_model.bert.encoder.layer.13.attention.output.dense.weight', 'q_model.bert.encoder.layer.23.attention.output.dense.bias', 'p_model.bert.encoder.layer.9.attention.self.value.bias', 'p_model.bert.encoder.layer.7.attention.self.value.weight', 'p_model.bert.encoder.layer.15.intermediate.dense.weight', 'q_model.bert.encoder.layer.3.output.dense.bias', 'p_model.bert.encoder.layer.17.attention.self.key.bias', 'q_model.bert.encoder.layer.9.intermediate.dense.bias', 'p_model.bert.encoder.layer.6.attention.output.LayerNorm.weight', 'q_model.bert.encoder.layer.1.intermediate.dense.bias', 'p_model.bert.encoder.layer.9.intermediate.dense.

bestmodel 체크포인트로 모델과 Trainer가 로드되었습니다.
BM25로 100개를 가져온 뒤 Dense retrieval로 40개의 문서를 찾습니다.
Data route: /data/ephemeral/data/train_dataset
Test data route: /data/ephemeral/data/test_dataset
Wiki route: /data/ephemeral/data/wikipedia_documents.json
Data path: ./bm25_retrieval_result


Token indices sequence length is longer than the specified maximum sequence length for this model (1131 > 512). Running this sequence through the model will result in indexing errors


File loaded successfully
위키 컨텍스트의 수: 56737


sequential reranking: 100%|██████████| 600/600 [29:10<00:00,  2.92s/it]


Reranked result saved to ./reranked/reranked_result.pkl


In [1]:
import torch

# GPU 메모리 캐시를 해제
torch.cuda.empty_cache()

# CUDA 메모리 통계 확인 (optional)
print(torch.cuda.memory_summary())

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |

In [2]:
import pickle

# 저장된 reranked_result 로드
load_path = "./reranked/reranked_result.pkl"

with open(load_path, 'rb') as f:
    reranked_result = pickle.load(f)

print("Reranked result loaded successfully")

Reranked result loaded successfully


In [3]:
from MRC import Extraction_based_MRC
model = Extraction_based_MRC()
model.train() #model.train()은 기존에 주어진 데이터셋을 활용하여 훈련합니다. model.train(train_dataset, eval_dataset)으로 본인이 직접 지정한 데이터를 훈련할 수도 있습니다.
# 이때, train 데이터셋의 형태는 load_from_disk(기존에 주어진 데이터경로)['train'], eval 데이터셋의 형태는 load_from_disk(기존에 주어진 데이터경로)['validation']의 형태여야 합니다.
# 모델은 Extraction_based_MRC_outputs + model_name의 폴더로 저장됩니다.
# 모델은 학습이 끝나면 가장 낮은 validaiton loss를 기준으로 저장됩니다.
model.inference(reranked_result)
# retrieval을 통해 얻어낸 결과를 predict_reuslt에 json파일 형태로 저장합니다.
"""inference는 반드시
DatasetDict({
    validation: Dataset({
        features: ['context', 'id', 'question'],
        num_rows: 600
    })
}) 의 형태를 따라야 합니다.
"""



[nltk_data] Downloading package punkt to
[nltk_data]     /data/ephemeral/home/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Some weights of the model checkpoint at snunlp/KR-ELECTRA-discriminator were not used when initializing ElectraForQuestionAnswering: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForQuestionAnswering were 

Using cuda_amp half precision backend
Map: 100%|██████████| 5936/5936 [00:07<00:00, 759.62 examples/s]


train_dataset을 넣지 않아 기존에 주어진 train dataset으로 학습합니다.


Map: 100%|██████████| 240/240 [00:00<00:00, 294.37 examples/s]
***** Running training *****
  Num examples = 5936
  Num Epochs = 4
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 744
  Number of trainable parameters = 108492290
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


eval_dataset을 넣지 않아 기존에 주어진 eval_dataset으로 평가합니다.


...

Saving model checkpoint to ./Extraction_based_MRC_outputs_KR-ELECTRA-discriminator/checkpoint-744
Configuration saved in ./Extraction_based_MRC_outputs_KR-ELECTRA-discriminator/checkpoint-744/config.json
Model weights saved in ./Extraction_based_MRC_outputs_KR-ELECTRA-discriminator/checkpoint-744/pytorch_model.bin
tokenizer config file saved in ./Extraction_based_MRC_outputs_KR-ELECTRA-discriminator/checkpoint-744/tokenizer_config.json
Special tokens file saved in ./Extraction_based_MRC_outputs_KR-ELECTRA-discriminator/checkpoint-744/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./Extraction_based_MRC_outputs_KR-ELECTRA-discriminator/checkpoint-372 (score: 1.5890638828277588).
Map: 100%|██████████| 600/600 [00:39<00:00, 15.35 examples/s]
The following columns in the test set don't have a corresponding argument in `ElectraForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping

post_process_function을 실행합니다.


100%|██████████| 600/600 [02:35<00:00,  3.85it/s]

Dataset({
    features: ['context', 'id', 'question'],
    num_rows: 600
})
output을 id : answers 형태의 json파일로 내보냅니다. 결과는 model.extraction_mrc_results로 확인할 수 있습니다.


"inference는 반드시\nDatasetDict({\n    validation: Dataset({\n        features: ['context', 'id', 'question'],\n        num_rows: 600\n    })\n}) 의 형태를 따라야 합니다.\n"

In [2]:
model.load_model() # 체크포인트의 제일 마지막 폴더를 따라가서 trainer_state.json을 열고 거기서 best_model_checkpoint의 경로를 통해 모델을 불러옵니다.
# 불러온 모델은 inference할 수 있습니다.

[nltk_data] Downloading package punkt to
[nltk_data]     /data/ephemeral/home/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForQuestionAnswering: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClass

가장 마지막 체크포인트로 모델과 Trainer가 로드되었습니다.


In [3]:
import All_dataset
from arguments import Extraction_based_MRC_arguments
datas = All_dataset.prepare_dataset(Extraction_based_MRC_arguments)
# 이처럼 All_dataset.prepare_dataset(argument에서 불러온 모델의 arg)를 통해 각 모델에서 쓰이는 데이터들을 볼 수 있습니다.
# 아무것도 주지 않으면 기본으로 주어진 데이터를 바탕으로 데이터를 전처리합니다.
# get_mrc_train_dataset(여러분이 바꾼 데이터) 를 통해 바꾼 데이터를 전처리할 수도 있습니다.
# train 데이터셋의 형태는 load_from_disk(기존에 주어진 데이터경로)['train'], eval 데이터셋의 형태는 load_from_disk(기존에 주어진 데이터경로)['validation']의 형태여야 합니다.
train = datas.get_mrc_train_dataset()
eval = datas.get_mrc_eval_dataset()



loading configuration file config.json from cache at /data/ephemeral/home/.cache/huggingface/hub/models--klue--bert-base/snapshots/77c8b3d707df785034b4e50f2da5d37be5f0f546/config.json
Model config BertConfig {
  "_name_or_path": "klue/bert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

loading file vocab.txt from cache at /data/ephemeral/home/.cache/huggingface/hub/models--klue--bert-base/snapshots/77c8b3d707df785034b4e50f2da5d37be5f0f546/vocab.txt
loading file tokenizer.json

In [1]:
import torch

# GPU 메모리 캐시를 해제
torch.cuda.empty_cache()

# CUDA 메모리 통계 확인 (optional)
print(torch.cuda.memory_summary())

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |

In [2]:
from MRC import Llama_MRC_Inference


model_name = "Bllossom/llama-3.2-Korean-Bllossom-3B"
test_dataset_path = "/data/ephemeral/data/test_dataset"
llama_mrc = Llama_MRC_Inference(model_name)
predictions = llama_mrc.inference(test_dataset_path)


[nltk_data] Downloading package punkt to
[nltk_data]     /data/ephemeral/home/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


KeyError: 'llama'